In [16]:
'''
This is a snippet of code showing how to select the hyper-parameters
of a MKL method using boolean kernels
Author: Ivano Lauriola, ivano.lauriola@phd.unipd.it
'''
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm



#load data
print ('loading \'iris\' (multiclass) dataset...', end='')
from sklearn.datasets import load_iris
ds = load_iris()
X,Y = ds.data, ds.target

'''
WARNING: be sure that your matrix is not sparse! EXAMPLE:
from sklearn.datasets import load_svmlight_file
X,Y = load_svmlight_file(...)
X = X.toarray()
'''

print ('done')

loading 'iris' (multiclass) dataset...done


In [17]:
X.shape

(150, 4)

In [30]:
#preprocess data
print ('preprocessing data...', end='')
#boolean kernels can be applied on binary-valued data, i.e. {0,1}.
#in this example, we binarize a real-valued dataset
#in this example, we binarize a real-valued dataset
from MKLpy.preprocessing import binarization
binarizer = binarization.AverageBinarizer()
Xbin = binarizer.fit_transform(X,Y)
print ('done')

#compute normalized homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing monotone Conjunctive Kernels...', end='')
from MKLpy.metrics import pairwise
from MKLpy.preprocessing import kernel_normalization
#WARNING: the maximum arity of the conjunctive kernel depends on the number of active variables for each example,
# that is 4 in the case of iris dataset binarized
KL = [kernel_normalization(pairwise.monotone_conjunctive_kernel(Xbin, c=c)) for c in range(5)]
print ('done')

preprocessing data...done
computing monotone Conjunctive Kernels...done


In [31]:

#train/test KL split (N.B. here we split a kernel list directly)
from MKLpy.model_selection import train_test_split
KLtr,KLte,Ytr,Yte = train_test_split(KL, Y, test_size=.3, random_state=42)

#MKL algorithms
from MKLpy.algorithms import EasyMKL, KOMD	#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
from MKLpy.model_selection import cross_val_score, cross_val_predict
from sklearn.svm import SVC
import numpy as np
print ('tuning lambda for EasyMKL...', end='')
base_learner = SVC(C=10000)	#simil hard-margin svm
best_results = {}


tuning lambda for EasyMKL...

In [32]:

#MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD	#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
# #select the base-learner
# #MKL algorithms use a hard-margin as base learned (or KOMD in the case of EasyMKL). It is possible to define a different base learner
# from sklearn.svm import SVC
# base_learner = SVC(C=0.1)
# clf = EasyMKL(learner=base_learner)
# clf = clf.fit(KLtr,Ytr)
print ('training AverageMKL...', end='')
clf = AverageMKL().fit(KLtr,Ytr)	#a wrapper for averaging kernels
print ('done')
print(clf.weights)			#print the weights of the combination of base kernels
K_average = clf.ker_matrix	#the combined kernel matrix


print ('training EasyMKL...', end='')
clf = EasyMKL(lam=0.1).fit(KLtr,Ytr)		#combining kernels with the EasyMKL algorithm
#lam is a hyper-parameter in [0,1]
print ('done')
print (clf.weights)

# #evaluate the solution
# from sklearn.metrics import accuracy_score, roc_auc_score
# y_pred = clf.predict(KLte)					#predictions
# y_score = clf.decision_function(KLte)		#rank
# accuracy = accuracy_score(Yte, y_pred)
# roc_auc = roc_auc_score(Yte, y_score)
# print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))




training AverageMKL...done
[0.2 0.2 0.2 0.2 0.2]
training EasyMKL...done
[-9.71714556e-17  6.14493694e-02  1.64774518e-01  3.03677199e-01
  4.70098913e-01]


In [33]:
clf=AverageMKL()

In [34]:
#load data
print ('loading \'iris\' dataset...', end='')
from sklearn.datasets import load_iris
import numpy as np
ds = load_iris()
X,Y = ds.data, ds.target
classes = np.unique(Y)
print ('done [%d classes]' % len(classes))

'''
WARNING: be sure that your matrix is not sparse! EXAMPLE:
from sklearn.datasets import load_svmlight_file
X,Y = load_svmlight_file(...)
X = X.toarray()
'''

#compute homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing Homogeneous Polynomial Kernels...', end='')
from MKLpy.metrics import pairwise
KL = [pairwise.homogeneous_polynomial_kernel(X, degree=d) for d in range(1,4)]
KLconjuctive = [kernel_normalization(pairwise.monotone_conjunctive_kernel(Xbin, c=c)) for c in range(5)]
print ('done')


# #MKL algorithms
# from MKLpy.algorithms import EasyMKL
# print ('training EasyMKL...', end='')
# clf = EasyMKL(lam=0.1).fit(KL,Y)		#combining kernels with the EasyMKL algorithm
# #multiclass_strategy should be 'ovo' for one-vs-one decomposition strategy, and 'ova' for one-vs-all/rest strategy
# print ('done')

# print (clf.weights)

loading 'iris' dataset...done [3 classes]
computing Homogeneous Polynomial Kernels...done


In [35]:


#load data
print ('loading \'breast cancer\' dataset...', end='')
from sklearn.datasets import load_breast_cancer
ds = load_breast_cancer()
X,Y = ds.data, ds.target
print ('done')

'''
WARNING: be sure that your matrix is not sparse! EXAMPLE:
from sklearn.datasets import load_svmlight_file
X,Y = load_svmlight_file(...)
X = X.toarray()
'''

#preprocess data
print ('preprocessing data...', end='')
from MKLpy.preprocessing import normalization, rescale_01
X = rescale_01(X)	#feature scaling in [0,1]
X = normalization(X) #||X_i||_2^2 = 1

#train/test split
from sklearn.model_selection import train_test_split
Xtr,Xte,Ytr,Yte = train_test_split(X,Y, test_size=.25, random_state=42)
print ('done')

loading 'breast cancer' dataset...done
preprocessing data...done


In [36]:
pd.DataFrame(X), print(np.unique(Y))

[0 1]


(           0         1         2         3         4         5         6   \
 0    0.186819  0.008124  0.195765  0.130417  0.212891  0.283987  0.252112   
 1    0.350623  0.148599  0.335707  0.273452  0.158034  0.099095  0.111001   
 2    0.265229  0.172085  0.262692  0.198170  0.226784  0.190056  0.203944   
 3    0.068534  0.117710  0.076171  0.033569  0.264663  0.264676  0.184507   
 4    0.321793  0.079991  0.322352  0.249963  0.219853  0.177728  0.237001   
 ..        ...       ...       ...       ...       ...       ...       ...   
 564  0.293842  0.182614  0.289016  0.241245  0.224405  0.126078  0.243362   
 565  0.297467  0.299697  0.288727  0.226579  0.194918  0.123186  0.161273   
 566  0.276412  0.377192  0.270666  0.184042  0.174963  0.154426  0.131604   
 567  0.209616  0.215777  0.216436  0.154705  0.191330  0.256976  0.267753   
 568  0.039172  0.532851  0.030323  0.016900  0.000000  0.078996  0.000000   
 
            7         8         9   ...        20        21   

In [37]:
#compute homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing Homogeneous Polynomial Kernels...', end='')
from MKLpy.metrics import pairwise
KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(11)]
KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(11)]
print ('done')


computing Homogeneous Polynomial Kernels...done


In [38]:
#MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD	#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
print ('training AverageMKL...', end='')
clf = AverageMKL().fit(KLtr,Ytr)	#a wrapper for averaging kernels
print ('done')
print(clf.weights)			#print the weights of the combination of base kernels
K_average = clf.ker_matrix	#the combined kernel matrix


print ('training EasyMKL...', end='')
clf = EasyMKL(lam=0.1).fit(KLtr,Ytr)		#combining kernels with the EasyMKL algorithm
#lam is a hyper-parameter in [0,1]
print ('done')
print (clf.weights)

training AverageMKL...done
[0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909]
training EasyMKL...done
[-1.08387208e-24  1.35673700e-02  2.76971359e-02  4.30853034e-02
  6.02816730e-02  7.96701534e-02  1.01481855e-01  1.25820437e-01
  1.52689783e-01  1.82019528e-01  2.13686761e-01]


In [39]:
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
y_pred = clf.predict(KLte) #predictions
y_score = clf.decision_function(KLte) #rank
accuracy = accuracy_score(Yte, y_pred)
roc_auc = roc_auc_score(Yte, y_score)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))


#select the base-learner
#MKL algorithms use a hard-margin as base learned (or KOMD in the case of EasyMKL). It is possible to define a different base learner
from sklearn.svm import SVC
base_learner = SVC(C=0.1, kernel='rbf')
clf = EasyMKL(learner=base_learner)
clf = clf.fit(KLtr,Ytr)

Accuracy score: 0.923, roc AUC score: 0.988


clf.predict(KLte)

In [40]:
for lam in [0, 0.01, 0.1, 0.2, 0.9, 1]:	#possible lambda values for the EasyMKL algorithm
    #MKLpy.model_selection.cross_val_predict performs the cross validation automatically, it optimizes the accuracy
    #the counterpart cross_val_score optimized the roc_auc_score (use score='roc_auc')
    #WARNING: these functions will change in the next version
    scores = cross_val_predict(KLtr, Ytr, EasyMKL(learner=base_learner, lam=lam), n_folds=5, score='accuracy')
    acc = np.mean(scores)
    if not best_results or best_results['score'] < acc:
        best_results = {'lam' : lam, 'score' : acc}

In [41]:
#evaluation on the test set
from sklearn.metrics import accuracy_score
print ('done')
clf = EasyMKL(learner=base_learner, lam=best_results['lam']).fit(KLtr,Ytr)
y_pred = clf.predict(KLte)
accuracy = accuracy_score(Yte, y_pred)
print ('accuracy on the test set: %.3f, with lambda=%.2f' % (accuracy, best_results['lam']))

done
accuracy on the test set: 0.923, with lambda=0.00


In [50]:
import os

#   data_dir: main directory , data_only_drive: the big drive where everything is saved
# data only dir: main drive that has the
data_dir = os.getenv('FINANCE_DATA')
data_only_drive = '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2'  # external date only drive

In [51]:
os.listdir(data_only_drive)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2'